In [0]:
# %matplotlib inline

import os

import torch
import torchaudio
import requests
import matplotlib
import matplotlib.pyplot as plt
import IPython

matplotlib.rcParams['figure.figsize'] = [16.0, 4.8]

torch.random.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

SPEECH_URL = 'priyanka_chunk.wav'#"https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/source-16k/train/sp0307/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav"
SPEECH_FILE = 'priyanka_chunk.wav'#"_assets/speech.wav"

if not os.path.exists(SPEECH_FILE):
  os.makedirs('_assets', exist_ok=True)
  with open(SPEECH_FILE, 'wb') as file:
    file.write(requests.get(SPEECH_URL).content)
    
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = bundle.get_model().to(device)

waveform, sample_rate = torchaudio.load(SPEECH_FILE)
waveform = waveform.to(device)

if sample_rate != bundle.sample_rate:
  waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

with torch.inference_mode():
  features, _ = model.extract_features(waveform)

with torch.inference_mode():
  emission, _ = model(waveform)

class GreedyCTCDecoder(torch.nn.Module):
  def __init__(self, labels, ignore):
    super().__init__()
    self.labels = labels
    self.ignore = ignore

  def forward(self, emission: torch.Tensor) -> str:
    """Given a sequence emission over labels, get the best path string
    Args:
      emission (Tensor): Logit tensors. Shape `[num_seq, num_label]`.

    Returns:
      str: The resulting transcript
    """
    indices = torch.argmax(emission, dim=-1)  # [num_seq,]
    indices = torch.unique_consecutive(indices, dim=-1)
    indices = [i for i in indices if i not in self.ignore]
    return ''.join([self.labels[i] for i in indices])


decoder = GreedyCTCDecoder(
    labels=bundle.get_labels(),
    ignore=(0, 1, 2, 3),
)
transcript = decoder(emission[0])

print(transcript)
IPython.display.Audio(SPEECH_FILE)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\ACER\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-1b6ab2fd3910>", line 38, in <module>
    features, _ = model.extract_features(waveform)
  File "C:\Users\ACER\anaconda3\lib\site-packages\torchaudio\models\wav2vec2\model.py", line 80, in extract_features
    x = self.encoder.extract_features(x, lengths, num_layers)
  File "C:\Users\ACER\anaconda3\lib\site-packages\torchaudio\models\wav2vec2\components.py", line 468, in extract_features
    return self.transformer.get_intermediate_outputs(
  File "C:\Users\ACER\anaconda3\lib\site-packages\torchaudio\models\wav2vec2\components.py", line 417, in get_intermediate_outputs
    x = layer(x, attention_mask)
  File "C:\Users\ACER\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "C:\Users\ACER\an

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\ACER\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-1b6ab2fd3910>", line 38, in <module>
    features, _ = model.extract_features(waveform)
  File "C:\Users\ACER\anaconda3\lib\site-packages\torchaudio\models\wav2vec2\model.py", line 80, in extract_features
    x = self.encoder.extract_features(x, lengths, num_layers)
  File "C:\Users\ACER\anaconda3\lib\site-packages\torchaudio\models\wav2vec2\components.py", line 468, in extract_features
    return self.transformer.get_intermediate_outputs(
  File "C:\Users\ACER\anaconda3\lib\site-packages\torchaudio\models\wav2vec2\components.py", line 417, in get_intermediate_outputs
    x = layer(x, attention_mask)
  File "C:\Users\ACER\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "C:\Users\ACER\an

TypeError: object of type 'NoneType' has no len()

In [2]:
print("Sample Rate:", bundle.sample_rate)

print("Labels:", bundle.get_labels())

print(model.__class__)

IPython.display.Audio(SPEECH_FILE)

fig, ax = plt.subplots(len(features), 1, figsize=(16, 4.3 * len(features)))
for i, feats in enumerate(features):
  ax[i].imshow(feats[0].cpu())
  ax[i].set_title(f"Feature from transformer layer {i+1}")
  ax[i].set_xlabel("Feature dimension")
  ax[i].set_ylabel("Frame (time-axis)")
plt.tight_layout()
plt.show()

plt.imshow(emission[0].cpu().T)
plt.title("Classification result")
plt.xlabel("Frame (time-axis)")
plt.ylabel("Class")
plt.show()
print("Class labels:", bundle.get_labels())

Sample Rate: 16000
Labels: ('<s>', '<pad>', '</s>', '<unk>', '|', 'E', 'T', 'A', 'O', 'N', 'I', 'H', 'S', 'R', 'D', 'L', 'U', 'M', 'W', 'C', 'F', 'G', 'Y', 'P', 'B', 'V', 'K', "'", 'X', 'J', 'Q', 'Z')


In [6]:
from pytube import YouTube
import os
def vid_down():
    link = input('Video link -->')
    destination = str(input(">> ")) or '.'
    yt = YouTube(link)
    video = yt.streams.filter(only_audio=True).first()
    out_file = video.download(output_path=destination)
  
    base, ext = os.path.splitext(out_file)
    new_file = base + '.wav'
    os.rename(out_file, new_file)

vid_down()

Video link -->https://www.youtube.com/watch?v=Wibxjrxf5ko&list=PLeo1K3hjS3uu7CxAacxVndI4bE_o3BDtO&index=10
>> 


In [30]:
lines = 0
a = 0
'''splitted = transcript.split('|')
a = 0
for i in range(0,len(splitted),3):
    text = splitted[i:i+3]
    for i in text:
        print(i,end=' ')
    print('\n')
   ''' 

for i in range(0,len(splitted),3):
    lines+=1
    text = splitted[i:i+3]
    with open('D:\python training\Opencv\Resources\sampletrans.vtt','a') as src:
        if lines==1:
            src.write('WEBVTT')
            src.write('\n\n')
        src.write(str(lines))
        src.write('\n')
        if len(str(a)) == 2:
            src.write('00:00:'+str(a)+'.000 --> '+'00:00:')
            a +=3
            src.write(str(a)+'.000')
            src.write('\n')
            for i in text:
                src.write(i)
                src.write(' ')
            src.write('\n')
        else:
            src.write('00:00:0'+str(a)+'.000 --> '+'00:00:0')
            a+=3
            src.write(str(a)+'.000')
            src.write('\n')
            for i in text:
                src.write(i)
                src.write(' ')
            src.write('\n')
        src.write('\n')
        src.close()


In [45]:
from pytube import YouTube

In [46]:
k = YouTube('https://www.youtube.com/watch?v=YmDaqXMIoeY&list=PLeo1K3hjS3uu7CxAacxVndI4bE_o3BDtO&index=17')

In [48]:
z = k.streams.filter(progressive=True).all()
z[1].download()

<ipython-input-48-81e69ca5fd34>:1: DeprecationWarning: Call to deprecated function all (This object can be treated as a list, all() is useless).
  z = k.streams.filter(progressive=True).all()


'C:\\Users\\ACER\\GPU bench-marking with image classification  Deep Learning Tutorial 17 (Tensorflow20 Python).mp4'

In [33]:
k.captions

C:\Users\ACER\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'a.en': <Caption lang="English (auto-generated)" code="a.en">}

In [43]:
xml_captions = k.captions['a.en']
srt_format = xml_captions.xml_caption_to_srt(xml_captions.xml_captions)
srt_format

C:\Users\ACER\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


KeyError: 'start'

In [8]:
from pytube import YouTube
import os
def vid_down():
    link = input('Video link -->')
    destination = str(input(">> ")) or '.'
    yt = YouTube(link)
    video = yt.streams.filter(only_audio=True).first()
    out_file = video.download(output_path=destination)
    print(out_file)

vid_down()

Video link -->https://www.youtube.com/watch?v=EDQ1dmFEGiI
>> .
C:\Users\ACER\.\Gif video clip.mp4


In [16]:
from pydub import AudioSegment
def chunk():
    audio_c_path = 'C:/Users/ACER/priyanka voice.wav' #input('Audio path --->')
    sound = AudioSegment.from_file(audio_c_path)
    #half = len(sound) // 2
    chun_k = sound[20000:100000]
    print('Exported at ',chun_k.export('C:/Users/Acer/priyanka_chunk.wav',format='wav'))
    print(len(sound))
    
chunk()


Exported at  <_io.BufferedRandom name='C:/Users/Acer/priyanka_chunk.wav'>
1080842
